In [ ]:
import torch
import numpy as np
import torch.optim as optim
from torch.utils import data
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import pandas as pd
from sklearn.metrics import roc_auc_score
import datetime
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from io import StringIO
import ffmpeg
import pdb
from functions import create_pytorch_dataset, create_multimodal_pytorch_dataset
from functions import get_window_metrics
from functions import get_frame_metrics
from functions import animate

In [ ]:
# Lets load the H%PY dataset into a pytorch dataset class.Please see 
# dataset_creator on how to generate the H5PY file. 

# Name of the H5PY dataset 
dset = ["Edits/ONI_IR", 'Edits/ONI_Depth'] #where the orginal data is stored 
name = ["ONI_IR_Edit", "ONI_IR_Depth"] # name of the h5py file
path = "H5Data\Data_set-{}-imgdim64x64.h5".format(name) # location of the h5py file
# this will also window the data at a set size, and with the set stride 

window_len = 8
stride = 1

Test_Dataset, test_dataloader, Train_Dataset, train_dataloader = create_multimodal_pytorch_dataset(name, dset, path, window_len, stride)


In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        # define layers 
        
        # first layer
        self.ec1 = nn.Conv3d(1, 16, (5, 3, 3), stride=1, padding=(2, 1, 1),)
        self.em1 = nn.MaxPool3d((1, 2, 2), return_indices=True)
        self.ed1 = nn.Dropout3d(p=0.25)
        # second layer
        self.ec2 = nn.Conv3d(16, 8, (5, 3, 3), stride=1, padding=(2, 1, 1))
        self.em2 = nn.MaxPool3d((2, 2, 2), return_indices=True)
        self.ed2 = nn.Dropout3d(p=0.25)
        # third layer
        self.ec3 = nn.Conv3d(8, 8, (5, 3, 3), stride=1, padding=(2, 1, 1))
        self.em3 = nn.MaxPool3d((2, 2, 2), return_indices=True)
        # encoding done, time to decode
        self.dc1 = nn.ConvTranspose3d(8, 8, (5, 3, 3), stride=1, padding=(2, 1, 1))
        self.dm1 = nn.MaxUnpool3d((2, 2, 2))
        # inverse of 2nd Conv
        self.dc2 = nn.ConvTranspose3d(8, 8, (5, 3, 3), stride=1, padding=(2, 1, 1))
        self.dm2 = nn.MaxUnpool3d((2, 2, 2))
        # inverse of 1st Conv
        self.dc3 = nn.ConvTranspose3d(8, 16, (5, 3, 3), stride=1, padding=(2, 1, 1))
        self.dm3 = nn.MaxUnpool3d((1, 2, 2))
        # final inverse
        self.dc4 = nn.ConvTranspose3d(16, 1, (5, 3, 3), stride=1, padding=(2, 1, 1))

        # first layer
        self.2ec1 = nn.Conv3d(1, 16, (5, 3, 3), stride=1, padding=(2, 1, 1),)
        self.2em1 = nn.MaxPool3d((1, 2, 2), return_indices=True)
        self.2ed1 = nn.Dropout3d(p=0.25)
        # second layer
        self.2ec2 = nn.Conv3d(16, 8, (5, 3, 3), stride=1, padding=(2, 1, 1))
        self.2em2 = nn.MaxPool3d((2, 2, 2), return_indices=True)
        self.2ed2 = nn.Dropout3d(p=0.25)
        # third layer
        self.2ec3 = nn.Conv3d(8, 8, (5, 3, 3), stride=1, padding=(2, 1, 1))
        self.2em3 = nn.MaxPool3d((2, 2, 2), return_indices=True)
        # encoding done, time to decode
        self.2dc1 = nn.ConvTranspose3d(8, 8, (5, 3, 3), stride=1, padding=(2, 1, 1))
        self.2dm1 = nn.MaxUnpool3d((2, 2, 2))
        # inverse of 2nd Conv
        self.2dc2 = nn.ConvTranspose3d(8, 8, (5, 3, 3), stride=1, padding=(2, 1, 1))
        self.2dm2 = nn.MaxUnpool3d((2, 2, 2))
        # inverse of 1st Conv
        self.2dc3 = nn.ConvTranspose3d(8, 16, (5, 3, 3), stride=1, padding=(2, 1, 1))
        self.2dm3 = nn.MaxUnpool3d((1, 2, 2))
        # final inverse
        self.2dc4 = nn.ConvTranspose3d(16, 1, (5, 3, 3), stride=1, padding=(2, 1, 1))

    def forward(self, x1, x2):
        # *** start of encoder
        x1 = x1.permute(1, 0, 2, 3, 4)  # reorder to have correct dimensions
        # (batch_size, chanels, depth, width, height)
        _ec1 = F.relu(self.ec1(x1))
        _em1, i1 = self.em1(_ec1)
        _ec1 = self.ed1(_ec1)
        # second layer
        _ec2 = F.relu(self.ec2(_em1))
        _em2, i2 = self.em2(_ec2)
        _em2 = self.ed2(_em2)
        # third layer
        _ec3 = F.relu(self.ec3(_em2))
        _em3, i3 = self.em3(_ec3)

        x2 = x2.permute(1, 0, 2, 3, 4)  # reorder to have correct dimensions
        # (batch_size, chanels, depth, width, height)
        _2ec1 = F.relu(self.2ec1(x2))
        _2em1, i1 = self.2em1(_2ec1)
        _2ec1 = self.2ed1(_2ec1)
        # second layer
        _2ec2 = F.relu(self.2ec2(_2em1))
        _2em2, i2 = self.2em2(_2ec2)
        _2em2 = self.2ed2(_2em2)
        # third layer
        _2ec3 = F.relu(self.2ec3(_2em2))
        _2em3, i3 = self.2em3(_2ec3)

        # combined modalaties here 
        combo1 = (_2ec3*0.5) * _em3
        combo2 = _2ec3 * (_em3*0.5)


        # print("====== Encoding Done =========")
        # *** encoding done, time to decode
        _dc1 = F.relu(self.dc1(combo1))
        _dm1 = self.dm1(_dc1, i3, output_size=_em2.size())
        # second layer
        _dc2 = F.relu(self.dc2(_dm1))
        _dm2 = self.dm2(_dc2, i2)
        # third layer
        _dc3 = F.relu(self.dc3(_dm2))
        _dm3 = self.dm3(_dc3, i1)

        re_x1 = torch.tanh(self.2dc4(combo2))
        # *** encoding done, time to decode
        _2dc1 = F.relu(self.2dc1(_em3))
        _2dm1 = self.2dm1(_2dc1, i3, output_size = _2em2.size())
        # second layer
        _2dc2 = F.relu(self.2dc2(_2dm1))
        _2dm2 = self.2dm2(_2dc2, i2)
        # third layer
        _2dc3 = F.relu(self.2dc3(_2dm2))
        _2dm3 = self.2dm3(_2dc3, i1)

        re_x2 = torch.tanh(self.2dc4(_2dm3))

        return re_x1, re_x2

In [ ]:
# Now lets train our model

# prepare for GPU training 
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
torch.cuda.empty_cache()

# and lets set the hyperparameters! 

dropout = 0.25
learning_rate = 0.0002
num_epochs = 20
chunk_size = 64
forward_chunk = 8 
forward_chunk_size = 8 # this is smaller due to memory constrains 

# select which model - you could load your own or put it in the function above
model = Autoencoder().to(device)

loss_fn = nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate))

In [ ]:

def train_model(filepath):
    model.train()
    for epoch in range(num_epochs):
        val_loss = 0
        for i, sample in enumerate(train_dataloader):
            x_modality = []
            y_modality = []
            for j, modalities in enumerate(sample):
                if j == 1: # skip labels
                    continue
                sample1 = x_modality[0]
                sample2 = x_modality[1]
                # label = y_modality[modality] # dont need label for training
                sample = sample.to(device, dtype=torch.float)
                # split sample into smaller sizes due to GPU memory constraints
                chunks = torch.split(sample, chunk_size, dim=1)
                for chunk in chunks:
                    output = model(chunk)
                    output = output.to(device).permute(1, 0, 2, 3, 4)
                    model.zero_grad()
                    loss = loss_fn(output, chunk)
                    # ===================backward====================
                    # Getting gradients w.r.t. parameters
                    loss.backward()
                    # Updating parameters
                    optimizer.step()
                    # Clear gradients w.r.t. parameters
                    optimizer.zero_grad()
                    torch.cuda.empty_cache()
                        
                # ===================log========================
                print("epoch [{}/{}], loss:{:.4f}".format(epoch + 1, num_epochs, loss.item()))
                torch.save(model.state_dict(), filepath) # save the model each epoch at location filepath
        

In [ ]:
def foward_pass(path):
    model.load_state_dict(torch.load(path)) # load a saved model 
    model.eval()
    frame_stats = [] 
    window_stats = [] 

    with torch.no_grad():
        print("foward pass occuring")
        # just forward pass of model on test dataset
        for i, sample in enumerate(test_dataloader):
            x_modality = []
            y_modality = []
            for j, modalities in enumerate(sample):
                if j == 0:
                    x_modality = modalities
                if j == 1:
                    y_modality = modalities
                    
            for modality in range(len(x_modality)):
                torch.cuda.empty_cache()
                sample = sample.to(device, dtype=torch.float)
                chunks = torch.split(sample, forward_chunk, dim=1)
                recon_vid = []
                for chunk in chunks:
                    output = model(chunk)
                    output = output.to(device).permute(1, 0, 2, 3, 4)
                    recon_vid.append(output)
                    torch.cuda.empty_cache()
    
                output = torch.cat(recon_vid, dim=1)
                # convert tensors to numpy arrays for easy manipluations
                sample = sample.data.cpu().numpy()
                output = output.data.cpu().numpy()
                labels = labels.data.cpu().numpy()

                frame_mean, frame_std, frame_labels = get_frame_metrics(output, sample, labels, window_len)
                mean_window_error, std_window_error, window_labels = get_window_metrics(output, sample, labels, window_len)
                frame_stats.append([frame_mean, frame_std, frame_labels])
                window_stats.append([mean_window_error, std_window_error, window_labels])
                    '''
                    if j % 50 == 0:
                        animate(sample[0, :, :, :, :], output[0, :, :, :, :], frame_mean, dset, start_time)
                    '''
                    

    return(frame_stats, window_stats)

In [ ]:
start_time = str(datetime.datetime.today().strftime("%Y-%m-%d-%H-%M-%S"))
print(start_time)
filepath = (
    "Models\\"
    + dset 
    + 'RegularLoss'
    + start_time
)
#filepath = 'Models\Edits\ONI_Depth2020-11-26-10-07-38'
print(filepath)
train_model(filepath)
import functions
from functions import animate


frame_stats, window_stats = foward_pass(filepath)

In [ ]:
import functions
from functions import get_total_performance_metrics
from importlib import reload
reload(functions)
from functions import get_total_performance_metrics

get_total_performance_metrics(frame_stats, window_stats, window_len)


#get_total_performance_metrics(originals, reconstruced, testing_labels, window_len)